# Pregunta 2
Estudiante Jorge Pérez Facuse

Partamos definiendo las clases RSAReceiver y Sender

Obtengamos de INTERNET > https://medium.com/@prudywsh/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb una implementacion para obtener numeros primos grandes aleatorios

In [61]:
from random import randrange, getrandbits
def _is_prime(n, k=128):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True
def _generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p
def _generate_prime_number(length=1024):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not _is_prime(p, 128):
        p = _generate_prime_candidate(length)
    return p


Ahora una implementacion para encontrar un coprimo y una para convertir de ints a bytes sin darle el length del arreglo https://stackoverflow.com/questions/70446319/python-integer-to-bytes-object-smallest-possible-size

In [82]:
import math

def _get_coprime(N):
    i = 2
    while i < N:
        mcd = math.gcd(i,N)
        if mcd == 1:
            result = i
            return result
        i += 1

def _int_to_bytes(integer_in: int) -> bytes:
    """Convert an integer to bytes"""
    # Calculates the least amount of bytes the integer can be fit into
    length = math.ceil(math.log(integer_in)/math.log(256))

    return integer_in.to_bytes(length, 'big')

In [100]:
class RSAReceiver:

    def __init__(self, bit_len):
        self.P = _generate_prime_number(bit_len //2)
        self.Q = _generate_prime_number(bit_len //2)
        self.N = self.P * self.Q
        self.phi = (self.P - 1) * (self.Q - 1)
        #print(self.N.bit_length())
        self.d = _get_coprime(self.phi)
        self.e = pow(self.d, -1, self.phi)

    
    def get_public_key(self):
        key = bytearray()
        e_bytes = _int_to_bytes(self.e)
        N_bytes = _int_to_bytes(self.N)
        key += len(e_bytes).to_bytes(4, 'big')
        key += e_bytes
        key += len(N_bytes).to_bytes(4, 'big')
        key += N_bytes
        return key

    def decrypt(self, ciphertext):
        c = int.from_bytes(ciphertext, 'big')
        m = _int_to_bytes(pow(c, self.d, self.N))
        return m.decode('utf-8')

In [101]:
reciver = RSAReceiver(2048)

In [102]:
key = reciver.get_public_key()

In [94]:
len(key)

520

In [57]:
class RSASender:
    def __init__(self, public_key):
        self.public_key = public_key
        self.e_len = int.from_bytes(public_key[0:4], 'big')
        self.e = int.from_bytes(public_key[4:4+self.e_len], 'big')
        self.N_len = int.from_bytes(public_key[4+self.e_len: 8+self.e_len], 'big')
        self.N = int.from_bytes(public_key[8+self.e_len:], 'big')
        self.n = (self.N.bit_length() -1) // 8
        """print(self.public_key)
        print(self.e_len)
        print(self.e)
        print(self.N_len)
        print(self.N)"""


    def encrypt(self, message):
        byte_message = bytearray(message, 'utf-8')
        enc_message = bytearray()
        i = 0
        while i < len(byte_message):
            b = byte_message[i: i+self.n] # Tomamos un bloque de size n
            #print(i)
            a = int.from_bytes(b, 'big') # convertimos b en un int
            c = pow(a, self.e, self.N)
            c = c.to_bytes(self.n + 1, 'big')
            enc_message += c
            i += self.n

        
        return enc_message


Finalmente testeamos nuestro SOLUTION

In [103]:
from base64 import b64encode, b64decode
public_key = b64decode('AAAAQQGHaihgiufnjzyLXufDjUCGuaHrsUL+hCF/pMFHPoh+ZVi/2bMFh6oelzElVklsJ9mglyQjJIKAb1JB9mvtaEkLAAAAQQHIuF+wIJw6uzq8uXpW/QmsNjtBJ8HCJJcu2h7sDX18nc2qWYDWTfMiXPmPRvhkkz4A0oXTAMDP9xsxUIjYQNsx')
text = (
    'Being open source means anyone can independently review '
    'the code. If it was closed source, nobody could verify the '
    'security. I think it’s essential for a program of this '
    'nature to be open source.'
)
sender = RSASender(key)
cipher = sender.encrypt(text)
print(b64encode(cipher))
#b64encode(cipher) == b'ALwPm7JXWbqGeIflV8PYgprs6mSgCH2Ydy0rgvFolzY0mczKItlPSHueL54uvDJXIz9pXoHZGAOPWVYYbcwRh3EBl8pi3MraUC2BBFUviMPFwNMwza/QMd5DNG9tH8doHlLRRt+15wLrsIE+m5T8fuM4HHixSNcEoOdN8T++q0PkzQDXL+UgbusiD3J+QPO59aqAB5HFcZ7P5U3fhFS8Qm1vLG8vlIulCby0jGLgjTtLUhFD/QhAof0y4F20gxedQDHwAOIrz6PEoBWnHmwLU0QNN0Rs542RvJ8BeEGhBDS5ZvD0/0Ix3ZqKT6HtP4ugfPD75/5LYGioJBwrg2DXbQucFj8='

197
b'kZPadSZ7i2tpOHdSO7YGeI1Aq4a2+sl1ZUAmdzy3Xu7LPGEqtpeLwPNgq2NVaR0ZMHOAzEhUKYJM0lP5vtKLlnfMXNcA8dldKaHzWi6nPaw3MZdi+W8NeQOWtcJpx4Ko0PJAIipu3jw1elnfaEg7pqHMwBUpmadx8sgcKj6wDvB9yaSCIUM9xC1U2hmJ/QlOhkieugw/9MjS8IpKH1ULC8Q+46Lya37Ls5ioMZoznxxbdg7IGHlSfGZ0jiH9eg7cMkKqJkqn7RLROLJgC4veLqNN5Uj6o+EDIQparx8yOcGxJ+Tes02GiKHv9oDMrDRXMzMsxMtxs+wtOYrSkkYxgQ=='


In [104]:
reciver.decrypt(cipher)

'Being open source means anyone can independently review the code. If it was closed source, nobody could verify the security. I think it’s essential for a program of this nature to be open source.'